# **Installing Packages**

In [1]:
removeFile = False
if removeFile == True:
    import shutil
    shutil.rmtree("/kaggle/working/anchor") #removes directory/folder

    import os

    file_to_remove = "/kaggle/working/anchor.zip"

    if os.path.exists(file_to_remove):
        os.remove(file_to_remove)
        print(f"{file_to_remove} removed successfully.")
    else:
        print(f"{file_to_remove} does not exist in the directory.")


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/criminal-psychological-data/Criminal Psychological Dataset CSV.csv


# **Importing Libraries**

In [3]:
from sklearn.model_selection import train_test_split # Import train_test_split function
import time #for calculating execution time
import xgboost as xgb

In [4]:
!pip install gdown

import gdown

runOriginal = False

if (runOriginal == False): #MODDED RUN
    # Replace 'FILE_ID' with the actual file ID of your zip folder
    file_id = '1gdXeAX6YeKop8xHC_CCJh745piebie9e'

    # Define the URL using the file ID
    url = f'https://drive.google.com/uc?id={file_id}'

    # Download the zip folder
    output = '/kaggle/working/anchor.zip'  # Replace 'filename' with your desired name
    gdown.download(url, output, quiet=False)

    import zipfile

    zip_path = '/kaggle/working/anchor.zip'  # Replace 'filename' with the name you used in the previous step
    extract_path = '/kaggle/working/'  # Replace with the path where you want to extract the contents

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    
    print()
    print("Downloaded and extracted modified anchor zip")
    print()
else: #ORIGINAL RUN
    # Replace 'FILE_ID' with the actual file ID of your zip folder
    file_id = '11wfnjytPrSvbfBpYmsAGx5lbybx8xTNx'

    # Define the URL using the file ID
    url = f'https://drive.google.com/uc?id={file_id}'

    # Download the zip folder
    output = '/kaggle/working/anchor.zip'  # Replace 'filename' with your desired name
    gdown.download(url, output, quiet=False)

    import zipfile

    zip_path = '/kaggle/working/anchor.zip'  # Replace 'filename' with the name you used in the previous step
    extract_path = '/kaggle/working/'  # Replace with the path where you want to extract the contents

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    
    print()
    print("Downloaded and extracted original anchor zip")
    print()

    
from anchor import utils
from anchor import anchor_tabular

Downloading...
From: https://drive.google.com/uc?id=1gdXeAX6YeKop8xHC_CCJh745piebie9e
To: /kaggle/working/anchor.zip
100%|██████████| 430k/430k [00:00<00:00, 93.0MB/s]



Downloaded and extracted modified anchor zip



# **Data Processing**

In [5]:
#reading and preparing datasets
org_ds = pd.read_csv('/kaggle/input/criminal-psychological-data/Criminal Psychological Dataset CSV.csv')

#converting all columns to lowercase to standardize uss
org_ds.columns = org_ds.columns.str.lower()

#dataset without non-numeric unwanted fields like date, time, patient number, zero change values
numeric_ds = org_ds.drop(['cid','cjdid','staffid','idate','intstrt','instrta','suindt','stjail','pre30jl','pre180jl','cdob',
                          'chethn1','crace1','c23cmin','c23cmdt','ag1sdin','sdinj30','intend','intenda','editdate','d2bi2pma',
                          'd2bi2pmb','d2bi2pmc','c18pdmcl','c18pdmcp','c18pdmca','c18spgmc','anytxpm','datadmit','entpro',
                          'compl','onset1','neworrec','environ','course','worst','worst2','txhispsy','txdralc','hospspec',
                          'hospmed','othprob3','whowith','currdx','lifedes1','lifedes2','lifedes3','lifetx3','lifedes4',
                          'lifetx4','lifedes5','lifetx8','screen6a','s61b','s61bs','c17cmdt'] , axis = 1)

print("After removing unimportant fields " , len(numeric_ds.columns))

#converting categorical values to integer by One Hot Encoding
numeric_ds = pd.get_dummies(numeric_ds, columns = ['clive1','job1','majsup1','othoff','othdrug','qtother1',
                                                             'othtx1','resptx1','a38spgmc','a40spsu','c20spsu','f29spsu',
                                                             'diff1','diff2','diff3','onset2','new2','othprob1','othprob2',
                                                             'medvit','alcdrug','freetime','ruleout','lifetx1','lifeage2',
                                                             'lifetx2','lifetx5','lifedes6','lifetx6','lifedes7','lifetx7','lifedes8'])

print("After converting categorical fields " , len(numeric_ds.columns))

# replace field that's entirely space (or empty) with NaN
numeric_ds = numeric_ds.replace(r'^\s*$', np.nan, regex=True)
    
for (columnName, columnData) in numeric_ds.iteritems():
    numeric_ds[columnName] = pd.to_numeric(numeric_ds[columnName])
    numeric_ds[columnName].fillna(numeric_ds[columnName].mean())
    numeric_ds[columnName] = numeric_ds[columnName].replace(np.nan, numeric_ds[columnName].mean())
    numeric_ds[columnName].astype(str).astype(float)
    

#numeric_ds = numeric_ds.select_dtypes(exclude=['object'])


#for (columnName, columnData) in numeric_ds.iteritems():
#    print(numeric_ds[columnName].value_counts())

#removing special characters from feature names added by One Hot Encoding
import re
# Change columns names ([LightGBM] Do not support special JSON characters in feature name.)
new_names = {col: re.sub(r'[^A-Za-z0-9_]+', '', col) for col in numeric_ds.columns}
new_n_list = list(new_names.values())
# [LightGBM] Feature appears more than one time.
new_names = {col: f'{new_col}_{i}' if new_col in new_n_list[:i] else new_col for i, (col, new_col) in enumerate(new_names.items())}
numeric_ds = numeric_ds.rename(columns=new_names)


#for (columnName, columnData) in numeric_ds.iteritems():
#    print(columnName)

#dataset without features of suicide ideation
ds_woSuicide = numeric_ds.drop(['othoff_SUICIDEATTEMPTS','sidelf','mini4','mini4a','mhsf28a','mhsf29a','gain43',
                                'gain43a','gain44','gain44a','cursuic1','cursuic2','cursuic3','cursuic4','passuic1',
                                'passuic2','passuic3','passuic4','currsuic','pastsuic','sciddia1','sciddi1a','scidsev2'], axis = 1)

print("Columns in original dataset without suicide ideation" , len(ds_woSuicide.columns))

#dataset with features of suicide ideation
ds_wSuicide = numeric_ds
print("Columns in original dataset with suicide ideation" , len(ds_wSuicide.columns))

#SHAP 27 features
rds_woSuicide = ds_woSuicide[['phosplf','scrns3b3','pdborder','mhsftot','scrn3md','suprob2','s65cs','alctype','age1coc',
                              'age1alc','new2_','age1tob','drgprb3','mhsf35a','hosptime','anxlf','mhsf28','s65a','d6mddlta',
                              'majsup','arstdrg','depend','drgprb2','scrns3b2','age','tcuds','c1cm30','sattlf']]

#SHAP 27 features
rds_wSuicide = ds_wSuicide[['mhsf28a','sidelf','gain44','phosplf','age1jl','job','age1coc','drvlic','gain44a','coc6m','aspdtot',
                            'alctype','age1tob','c7cm6m','mshf10cs','clive','gain53a','suprob2','pdborder','age1mj','drgprb6',
                            'scrns3c3','arstdrg','allmon','hosptime','gs5a','depend','sattlf']]

print("Columns in reduced dataset without suicide ideation" , len(rds_woSuicide.columns))
print("Columns in reduced dataset with suicide ideation" , len(rds_wSuicide.columns))

#print(len(rds_wSuicide.columns))

After removing unimportant fields  860
After converting categorical fields  2337


/tmp/ipykernel_32/1557377917.py:29: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for (columnName, columnData) in numeric_ds.iteritems():


Columns in original dataset without suicide ideation 2314
Columns in original dataset with suicide ideation 2337
Columns in reduced dataset without suicide ideation 28
Columns in reduced dataset with suicide ideation 28


In [6]:
print("Previous without suicide anchor reduced features:")
print("phosplf,scrns3b3,scrn3md,pdborder,mhsftot,age,age1coc,hosptime,scrns3b2,mhsf28,mhsf35a,depend,arstdrg,alctype,d6mddlta,age1tob,c1cm30,tcuds,drgprb3,sattlf")
      
print("Previous with suicide anchor reduced features:")
print("mhsf28a,sidelf,gain44,pdborder,phosplf,depend,age1tob,arstdrg,gs5a,coc6m,age1mj,age1jl,sattlf")

Previous without suicide anchor reduced features:
phosplf,scrns3b3,scrn3md,pdborder,mhsftot,age,age1coc,hosptime,scrns3b2,mhsf28,mhsf35a,depend,arstdrg,alctype,d6mddlta,age1tob,c1cm30,tcuds,drgprb3,sattlf
Previous with suicide anchor reduced features:
mhsf28a,sidelf,gain44,pdborder,phosplf,depend,age1tob,arstdrg,gs5a,coc6m,age1mj,age1jl,sattlf


In [7]:
#set random_state = 1 for reproducible outcomes
#DATASET WITHOUT SUICIDE IDEATION
#,'mhsf28a','mhsf29a'
randomState = 3
#REDUCED DATASET WITHOUT SUICIDE IDEATION
X_r = rds_woSuicide.drop(['sattlf'], axis=1) #X training features
y_r = rds_woSuicide.sattlf #y output features
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_r, y_r, test_size=0.2, random_state=randomState) # 80% training and 20% test

#REDUCED DATASET WITH SUICIDE IDEATION
X_rs = rds_wSuicide.drop(['sattlf'], axis=1) #X training features
y_rs = rds_wSuicide.sattlf #y output features
X_train_rs, X_test_rs, y_train_rs, y_test_rs = train_test_split(X_rs, y_rs, test_size=0.2, random_state=randomState) # 80% training and 20% test

# **XGB Model**

In [8]:
#XGB model for anchor
from sklearn.metrics import mean_absolute_error, accuracy_score, recall_score, f1_score

def XGB(xTrain, xTest, yTrain, yTest, x, pngName):
    # Create classifer objects
    xgbModel = xgb.XGBClassifier(objective="binary:logistic", learning_rate=0.3, 
                               booster='gbtree', max_depth=5, n_estimators=100, enable_categorical=False)
    # Fit
    xgbModel.fit(xTrain, yTrain)
    # Predict
    y_pred = xgbModel.predict(xTest)
    
    #Anchor Explainer-------------------------------------------------------------------------
    #Anchor explaination below
    # Create a DataFrame with the feature names
    feature_names = list(xTrain.columns)
    df = pd.DataFrame(xTrain, columns=feature_names)

    # Create an instance of the AnchorTabularExplainer
    explainer = anchor_tabular.AnchorTabularExplainer(
        class_names=['0', '1'],  # List of class names
        feature_names=feature_names,  # List of feature names
        categorical_names={},  # Dictionary of categorical feature names
        train_data=xTrain.values,  # DataFrame of training data
        discretizer='decile'  # discretization method
    )

    precision_val = 0
    coverage_val = 0
    test_precision_val = 0
    test_coverage_val = 0
    mae_val = 0
    acc_val = 0
    yPred_instances = []  # Store y_pred_instance values for each instance
    #explainer.save_to_file(
    # Open the file in write mode
    fileDir = '/kaggle/working/' + pngName + '.txt'
    with open(fileDir, 'w') as f:
        # Write text to the file
        for index in range (0, len(xTest)):
            print(index)
            print("", file=f)
            # Choose a prediction to explain
            idx = index #index of records in testing depends on test/train split. currently 0-70
            instance = xTest.iloc[[idx]]
            predict_fn = lambda x: xgbModel.predict(x)

            # Generate an explanation for the chosen prediction
            explanation = explainer.explain_instance(
                instance.values[0],
                predict_fn,
                threshold=0.95,
                delta=0.1,  # default is 0.1 - last good result on 0.3
                tau=0.15  # default is 0.15 - last good result on 0.1
            )
            
            y_pred_instance = predict_fn(instance.values)
            # Ensure y_pred_instance is a 1-dimensional array
            y_pred_instance = np.squeeze(y_pred_instance)

            # Get the corresponding true label from yTest and convert to 1-dimensional array
            y_true = np.array([yTest.iloc[idx]])

            mae = mean_absolute_error(y_true, y_pred_instance.reshape(1, -1))
            accuracy = accuracy_score(y_true, y_pred_instance.reshape(1, -1))

            acc_val = acc_val + accuracy
            mae_val = mae_val + mae
            # Store y_pred_instance for each instance
            yPred_instances.append(np.ravel(y_pred_instance))


            # Print the explanation
            print('Patient {}:'.format(idx+1), file=f)
            print('Prediction: {}'.format(y_pred_instance), file=f)
            #print('Explanation: {}'.format(explanation), file=f)
            print('Anchor: %s' % (' AND '.join(explanation.names())), file=f)
            print('Precision: %.2f' % explanation.precision(), file=f)
            print('Coverage: %.2f' % explanation.coverage(), file=f)
            #print('Mean Absolute Error (MAE): {}'.format(mae), file=f)
            print('Accuracy: {}'.format(accuracy), file=f)


            # Get test examples where the anchor applies
            fit_anchor = np.where(np.all(xTest.values[:, explanation.features()] == instance.values[0][explanation.features()], axis=1))[0]
            print('Anchor test precision: %.2f' % (np.mean(predict_fn(xTest.values[fit_anchor]) == predict_fn(instance.values))), file=f)
            print('Anchor test coverage: %.2f' % (fit_anchor.shape[0] / float(xTest.shape[0])), file=f)

            precision_val = precision_val + explanation.precision()
            coverage_val = coverage_val + explanation.coverage()
            test_precision_val = test_precision_val + np.mean(predict_fn(xTest.values[fit_anchor]) == predict_fn(instance.values))
            test_coverage_val = test_coverage_val + (fit_anchor.shape[0] / float(xTest.shape[0]))

            
    # The file is automatically closed when the `with` block is exited

    print("!FILE WRITING HAS BEEN COMPLETED!")
    num_instances = xTest.shape[0]
    print(num_instances)
    precision_val = precision_val / num_instances
    coverage_val = coverage_val / num_instances
    test_precision_val = test_precision_val / num_instances
    test_coverage_val = test_coverage_val / num_instances
    print("Average Precision: " , round(precision_val, 3))
    print("Average Coverage: " , round(coverage_val, 3))
    print("Average Test Precision: " , round(test_precision_val, 3))
    print("Average Test Coverage: " , round(test_coverage_val, 3))
    mae_val = mae_val / num_instances
    acc_val = acc_val / num_instances
    print("Average MAE: ", round(mae_val, 3))
    print("Accuracy: " , round(acc_val, 3))

    # Create a new yPred based on stored y_pred_instance values
    yPred_anchor = np.concatenate(yPred_instances)
    # Calculate and print overall recall and F1 score for entire y_true and yPred
    recall_val = recall_score(yTest, yPred_anchor)
    f1_score_val = f1_score(yTest, yPred_anchor)

    print("Recall: " , round(recall_val, 3))
    print("F1 Score: " , round(f1_score_val, 3))

    
    
    return y_pred

In [9]:
def timeChange(oldTime, newTime):
    percentageChange = ((newTime-oldTime)/oldTime) * 100
    if (percentageChange >= 0):
        print(f"{percentageChange:.2f} % increase in time. Not Good.")
    else:
        percentageChange = percentageChange * (-1)
        print(f"{percentageChange:.2f} % decrease in time. Good Effort.")
        
def displayTime(seconds, NameText):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    remaining_seconds = seconds % 60
    print(NameText)
    print(f"{hours} hours, {minutes} minutes, {remaining_seconds} seconds. (Total {seconds} seconds)")

In [10]:
#running ensemble methods for classifiers

start_time_noSuicide = time.time()

y_pred_ens_r = XGB(X_train_r, X_test_r, y_train_r, y_test_r, X_r, "No Suicide") #reduced dataset

end_time_noSuicide = time.time()
execution_time_noSuicide = end_time_noSuicide - start_time_noSuicide

displayTime(round(execution_time_noSuicide, 2) , "Execution time Without Suicide Only:")
timeChange(348.79 , round(execution_time_noSuicide, 2))

start_time_Suicide = time.time()

y_pred_ens_rs = XGB(X_train_rs, X_test_rs, y_train_rs, y_test_rs, X_rs, "With Suicide") #reduced with suicide ideation

end_time_Suicide = time.time()
execution_time_Suicide = end_time_Suicide - start_time_Suicide

displayTime(round(execution_time_Suicide, 2) , "Execution time With Suicide Only:")
timeChange(402.74 , round(execution_time_Suicide, 2))

execution_time_complete = end_time_Suicide - start_time_noSuicide

displayTime(round(execution_time_complete, 2) , "Execution time Total:")
timeChange(751.53 , round(execution_time_complete, 2))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
!FILE WRITING HAS BEEN COMPLETED!
71
Average Precision:  0.989
Average Coverage:  0.321
Average Test Precision:  1.0
Average Test Coverage:  0.25
Average MAE:  0.099
Accuracy:  0.901
Recall:  0.556
F1 Score:  0.588
Execution time Without Suicide Only:
0.0 hours, 3.0 minutes, 31.02000000000001 seconds. (Total 211.02 seconds)
39.50 % decrease in time. Good Effort.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
!FILE WRITING HAS BEEN COMPLETED!
71
Average Precision:  0.989
Average Coverage:  0.534
Average Test Precision:  1.0
Average Test Coverage:  0.585
Average MAE:  0.028
Accuracy:  0.972
Recall:  0.889
F1 Score:  0.889
Execution tim